# 📦 Python Dataclasses: Koniec z bałaganem w konfiguracji

Jako Data Scientist często musisz zarządzać dziesiątkami parametrów:
*   Learning Rate
*   Liczba drzew
*   Ścieżka do danych
*   Wymiar warstwy ukrytej

**Sposób amatorski (Słownik):**
Trudno się pisze, brak podpowiadania kodu (IntelliSense), łatwo o literówkę.

**Sposób profesjonalny (Dataclass):**
Klasa, która generuje się "sama" (nie trzeba pisać nudnego `__init__`). Wymusza typy danych i blokuje przypadkowe zmiany.

W tym notatniku stworzymy profesjonalną konfigurację treningu.

In [1]:
# Sposób STARY (Słownik)
config_dict = {
    "learning_rate": 0.001,
    "batch_size": 32,
    "architecture": "ResNet50"
}

def train_model_old(cfg):
    # Ryzyko: Musisz pamiętać dokładne nazwy kluczy
    # Jeśli wpiszesz cfg['batch_sz'], kod wywali się dopiero tutaj
    print(f"Trenuję {cfg['architecture']} z LR={cfg['learning_rate']}")

train_model_old(config_dict)
print("Działa, ale jest ryzykowne (brak podpowiadania kodu).")

Trenuję ResNet50 z LR=0.001
Działa, ale jest ryzykowne (brak podpowiadania kodu).


## Sposób Nowy: @dataclass

Użyjemy dekoratora `@dataclass`.
Zauważ, że definiujemy zmienne jak w C++ czy Java (z typami).
Dzięki temu, gdy napiszesz `config.`, edytor kodu od razu podpowie Ci dostępne opcje!

In [2]:
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    model_name: str
    learning_rate: float
    batch_size: int
    # Możemy ustawić wartości domyślne
    epochs: int = 10
    use_gpu: bool = True

# Tworzenie instancji (Wygląda jak zwykła klasa)
config = TrainingConfig(
    model_name="GPT-Mini",
    learning_rate=0.0005,
    batch_size=64
)

print("--- NASZA KONFIGURACJA ---")
print(config)

# Dostęp przez kropkę (To jest to, co kochają programiści)
print(f"\nModel: {config.model_name}")
print(f"Epoki (domyślnie): {config.epochs}")

--- NASZA KONFIGURACJA ---
TrainingConfig(model_name='GPT-Mini', learning_rate=0.0005, batch_size=64, epochs=10, use_gpu=True)

Model: GPT-Mini
Epoki (domyślnie): 10


## Bezpieczeństwo: Frozen (Zamrożenie)

W ML często zdarza się głupi błąd: przypadkiem nadpisujesz parametr w trakcie treningu.
`config.learning_rate = 0` (przez pomyłkę).

Możemy temu zapobiec, dodając `frozen=True`.
Wtedy obiekt staje się **niemodyfikowalny** (read-only).

In [3]:
@dataclass(frozen=True)
class SafeConfig:
    lr: float
    steps: int

# Tworzymy bezpieczną konfigurację
safe_conf = SafeConfig(lr=0.01, steps=1000)

print(f"LR przed zmianą: {safe_conf.lr}")

# Próba zmiany (To powinno wywołać błąd!)
try:
    safe_conf.lr = 0.5
except Exception as e:
    print("\n🚫 BŁĄD! Nie można zmienić wartości (I BARDZO DOBRZE):")
    print(e)

LR przed zmianą: 0.01

🚫 BŁĄD! Nie można zmienić wartości (I BARDZO DOBRZE):
cannot assign to field 'lr'


## Praktyczne zastosowanie w funkcji

Zobacz, jak czytelna staje się funkcja treningowa, gdy używamy `dataclass`.
W definicji funkcji używamy **Type Hinting** (`cfg: TrainingConfig`).
Dzięki temu każdy, kto używa Twojej funkcji, wie dokładnie, co ma tam wrzucić.

In [4]:
# Funkcja przyjmuje TYLKO obiekt typu TrainingConfig
def train_model_pro(cfg: TrainingConfig):
    if not cfg.use_gpu:
        print("⚠️ Ostrzeżenie: Trenujesz na CPU. Będzie wolno.")
    
    print("-" * 30)
    print(f"🚀 Start treningu: {cfg.model_name}")
    print(f"Parametry: Batch={cfg.batch_size}, LR={cfg.learning_rate}")
    
    # Symulacja pętli treningowej
    for i in range(cfg.epochs):
        # Tu byłaby logika treningu...
        pass
    
    print("✅ Koniec.")

# Użycie
my_conf = TrainingConfig(model_name="RandomForest", learning_rate=0.01, batch_size=128, epochs=3)
train_model_pro(my_conf)

------------------------------
🚀 Start treningu: RandomForest
Parametry: Batch=128, LR=0.01
✅ Koniec.


## 🧠 Podsumowanie: Kodowanie dla Ludzi

Komputer zrozumie wszystko, nawet bałagan w słowniku.
Ale kod piszesz dla **ludzi** (i dla siebie z przyszłości).

**Zalety Dataclasses w ML:**
1.  **Autouzupełnianie:** Piszesz `cfg.` i widzisz listę parametrów. Nie musisz pamiętać nazw.
2.  **Typy:** Wiesz, że `batch_size` to `int`, a nie `str`.
3.  **Immutability (frozen):** Masz pewność, że nikt nie zepsuł konfiguracji w połowie skryptu.

**Co dalej?**
Mamy fajną klasę `TrainingConfig`.
Ale co, jeśli chcemy wczytać tę konfigurację z pliku JSON albo YAML? Albo stworzyć konfigurację na podstawie nazwy pliku?
Tutaj wchodzą **@classmethod** i **@staticmethod**, którymi zajmiemy się w następnym notatniku.